[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/langchain-course/blob/main/chapters/07-lcel.ipynb)

#### LangChain Essentials Course

# LangChains Expression Language

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain's Expression Langauge (LCEL), abstracting a full chain and understanding how it will work. We'll provide examples for both OpenAI's `gpt-4o-mini` *and* Meta's `llama3.2` via Ollama!

In [1]:
!pip install -qU \
  langchain-core==0.3.33 \
  langchain-openai==0.3.3 \
  langchain-community==0.3.16 \
  langsmith==0.3.4 \
  docarray==0.40.0


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\karol\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


---

> ⚠️ We will be using OpenAI for this example allowing us to run everything via API. If you would like to use Ollama instead, check out the [Ollama LangChain Course](https://github.com/aurelio-labs/langchain-course/tree/main/notebooks/ollama).

---

---

> ⚠️ If using LangSmith, add your API key below:

In [2]:
import os
from getpass import getpass

# must enter API key
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "aurelioai-langchain-course-langsmith-openai"

---

## Traditional Chains vs LCEL

In this section we're going to dive into a basic example using the traditional method for building chains before jumping into LCEL. We will build a pipeline where the user must input a specific topic, and then the LLM will look and return a report on the specified topic. Generating a _research report_ for the user.

### Traditional LLMChain

The `LLMChain` is the simplest chain originally introduced in LangChain. This chain takes a prompt, feeds it into an LLM, and _optionally_ adds an output parsing step before returning the result.

Let's see how we construct this using the traditional method, for this we need:

* `prompt` — a `PromptTemplate` that will be used to generate the prompt for the LLM.
* `llm` — the LLM we will be using to generate the output.
* `output_parser` — an optional output parser that will be used to parse the structured output of the LLM.

In [3]:
from langchain import PromptTemplate

prompt_template = "Give me a small report on {topic}"

prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_template
)

For the LLM, we'll start by initializing our connection to the OpenAI API. We do need an OpenAI API key, which you can get from the [OpenAI platform](https://platform.openai.com/api-keys).

We will use the `gpt-4o-mini` model with a `temperature` of `0.0`:

In [4]:
import os
from getpass import getpass
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") \
    or getpass("Enter your OpenAI API key: ")

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.0,
)

In [5]:
llm_out = llm.invoke("Hello there")
llm_out

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'finish_reason': 'stop', 'logprobs': None}, id='run-d2f8a41d-daa0-4ea8-a60b-27622ead4066-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Then we define our output parser, this will be used to parse the output of the LLM. In this case, we will use the `StrOutputParser` which will parse the `AIMessage` output from our LLM into a single string.

In [6]:
from langchain.schema.output_parser import StrOutputParser

output_parser = StrOutputParser()

In [7]:
out = output_parser.invoke(llm_out)
out

'Hello! How can I assist you today?'

Through the `LLMChain` class we can place each of our components into a linear `chain`.

In [8]:
from langchain.chains import LLMChain

chain = LLMChain(prompt=prompt, llm=llm, output_parser=output_parser)

C:\Users\karol\AppData\Local\Temp\ipykernel_10636\2960353250.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(prompt=prompt, llm=llm, output_parser=output_parser)


Note that the `LLMChain` _was_ deprecated in LangChain `0.1.17`, the expected way of constructing these chains today is through LCEL, which we'll cover in a moment.

We can `invoke` our `chain`, providing a `topic` that we'd like to be researched.

In [9]:
result = chain.invoke("retrieval augmented generation")
result

{'topic': 'retrieval augmented generation',
 'text': '### Report on Retrieval-Augmented Generation (RAG)\n\n#### Introduction\nRetrieval-Augmented Generation (RAG) is an advanced approach in natural language processing (NLP) that combines the strengths of information retrieval and generative models. This technique enhances the capabilities of language models by allowing them to access external knowledge sources, thereby improving the accuracy and relevance of generated responses.\n\n#### Concept Overview\nRAG operates on the principle of integrating a retrieval mechanism with a generative model. The process typically involves two main components:\n\n1. **Retrieval Component**: This part of the system retrieves relevant documents or pieces of information from a large corpus based on a given query. It often employs techniques such as vector embeddings and similarity search to identify the most pertinent data.\n\n2. **Generative Component**: Once relevant information is retrieved, a gener

We can view a formatted version of this output using the `Markdown` display:

In [10]:
from IPython.display import display, Markdown

display(Markdown(result["text"]))

### Report on Retrieval-Augmented Generation (RAG)

#### Introduction
Retrieval-Augmented Generation (RAG) is an advanced approach in natural language processing (NLP) that combines the strengths of information retrieval and generative models. This technique enhances the capabilities of language models by allowing them to access external knowledge sources, thereby improving the accuracy and relevance of generated responses.

#### Concept Overview
RAG operates on the principle of integrating a retrieval mechanism with a generative model. The process typically involves two main components:

1. **Retrieval Component**: This part of the system retrieves relevant documents or pieces of information from a large corpus based on a given query. It often employs techniques such as vector embeddings and similarity search to identify the most pertinent data.

2. **Generative Component**: Once relevant information is retrieved, a generative model (often based on architectures like Transformers) processes this information to produce coherent and contextually appropriate text. The generative model can be fine-tuned to ensure that the output is not only relevant but also stylistically consistent with the desired output.

#### Advantages of RAG
- **Enhanced Knowledge Access**: By leveraging external databases or knowledge bases, RAG can provide more accurate and up-to-date information than models relying solely on pre-trained knowledge.
- **Improved Contextual Relevance**: The retrieval step allows the model to ground its responses in specific, relevant content, leading to more contextually appropriate outputs.
- **Scalability**: RAG systems can scale to incorporate vast amounts of information, making them suitable for applications requiring extensive knowledge bases.

#### Applications
RAG has a wide range of applications, including:
- **Question Answering**: Providing precise answers to user queries by retrieving relevant documents and generating responses based on that information.
- **Chatbots and Virtual Assistants**: Enhancing conversational agents with the ability to pull in real-time information from external sources.
- **Content Creation**: Assisting in generating articles, reports, or summaries by retrieving relevant data and synthesizing it into coherent text.

#### Challenges
Despite its advantages, RAG also faces several challenges:
- **Quality of Retrieved Information**: The effectiveness of the generative model heavily depends on the quality and relevance of the retrieved documents.
- **Complexity of Integration**: Combining retrieval and generation components can introduce complexity in system design and implementation.
- **Latency**: The retrieval process can add latency to response times, which may be a concern in real-time applications.

#### Conclusion
Retrieval-Augmented Generation represents a significant advancement in the field of NLP, merging the capabilities of information retrieval with generative modeling. By enabling models to access and utilize external knowledge, RAG enhances the relevance and accuracy of generated content, making it a powerful tool for various applications. As research and development in this area continue, we can expect further improvements in the efficiency and effectiveness of RAG systems.

That is a simple `LLMChain` using the traditional LangChain method. Now let's move onto LCEL.

## LangChain Expression Language (LCEL)

**L**ang**C**hain **E**xpression **L**anguage (LCEL) is the recommended approach to building chains in LangChain. Having superceeded the traditional methods with `LLMChain`, etc. LCEL gives us a more flexible system for building chains. The pipe operator `|` is used by LCEL to _chain_ together components. Let's see how we'd construct an `LLMChain` using LCEL.

In [11]:
lcel_chain = prompt | llm | output_parser

We can `invoke` this chain in the same way as we did before:

In [12]:
result = lcel_chain.invoke("retrieval augmented generation")
result

'### Report on Retrieval-Augmented Generation (RAG)\n\n#### Introduction\nRetrieval-Augmented Generation (RAG) is an innovative approach that combines the strengths of information retrieval and natural language generation. This method enhances the capabilities of language models by allowing them to access external knowledge sources, thereby improving the accuracy and relevance of generated responses.\n\n#### Concept Overview\nRAG operates on the principle of integrating a retrieval mechanism with a generative model. The process typically involves two main components:\n\n1. **Retrieval Component**: This part of the system retrieves relevant documents or pieces of information from a large corpus based on a given query. It uses techniques such as vector embeddings and similarity search to identify the most pertinent data.\n\n2. **Generation Component**: After retrieving the relevant information, the generative model (often based on architectures like Transformers) synthesizes a coherent a

The output format is slightly different, but the underlying functionality and content being output is the same. As before, we can view a formatted version of this output using the `Markdown` display:

In [13]:
display(Markdown(result))

### Report on Retrieval-Augmented Generation (RAG)

#### Introduction
Retrieval-Augmented Generation (RAG) is an innovative approach that combines the strengths of information retrieval and natural language generation. This method enhances the capabilities of language models by allowing them to access external knowledge sources, thereby improving the accuracy and relevance of generated responses.

#### Concept Overview
RAG operates on the principle of integrating a retrieval mechanism with a generative model. The process typically involves two main components:

1. **Retrieval Component**: This part of the system retrieves relevant documents or pieces of information from a large corpus based on a given query. It uses techniques such as vector embeddings and similarity search to identify the most pertinent data.

2. **Generation Component**: After retrieving the relevant information, the generative model (often based on architectures like Transformers) synthesizes a coherent and contextually appropriate response. This model leverages the retrieved data to enhance its output, ensuring that the generated text is not only fluent but also factually grounded.

#### Advantages
- **Enhanced Knowledge Access**: By incorporating external knowledge, RAG can provide more accurate and contextually relevant information than standalone generative models.
- **Dynamic Information Retrieval**: RAG systems can adapt to new information in real-time, making them suitable for applications where up-to-date knowledge is crucial.
- **Improved Contextual Understanding**: The retrieval process allows the model to better understand the context of the query, leading to more relevant and precise responses.

#### Applications
RAG has a wide range of applications, including:
- **Question Answering**: Providing accurate answers to user queries by retrieving relevant documents and generating responses based on them.
- **Chatbots and Virtual Assistants**: Enhancing conversational agents with the ability to pull in real-time information from various sources.
- **Content Creation**: Assisting writers and content creators by providing relevant data and context for generating articles, reports, or creative writing.

#### Challenges
Despite its advantages, RAG also faces several challenges:
- **Quality of Retrieved Information**: The effectiveness of the RAG model heavily depends on the quality and relevance of the retrieved documents. Poor retrieval can lead to inaccurate or misleading outputs.
- **Computational Complexity**: The dual process of retrieval and generation can be resource-intensive, requiring significant computational power and optimization.
- **Integration of Diverse Sources**: Ensuring that the model can effectively integrate and synthesize information from various types of sources can be complex.

#### Conclusion
Retrieval-Augmented Generation represents a significant advancement in the field of natural language processing, merging the capabilities of information retrieval with generative models. As research and development in this area continue to evolve, RAG has the potential to transform various applications, making them more intelligent and responsive to user needs. Future work will likely focus on improving retrieval accuracy, optimizing computational efficiency, and expanding the range of applications for this promising technology.

### How Does the Pipe Operator Work?

Before moving onto other LCEL features, let's take a moment to understand what the pipe operator `|` is doing and _how_ it works.

Functionality wise, the pipe tells you that whatever the _left_ side outputs will be fed as input into the _right_ side. In the example of `prompt | llm | output_parser`, we see that `prompt` feeds into `llm` feeds into `output_parser`.

The pipe operator is a way of chaining together components, and is a way of saying that whatever the _left_ side outputs will be fed as input into the _right_ side.

Let's make a basic class named `Runnable` that will transform our a provided function into a _runnable_ class that we will then use with the pipe `|` operator.

In [30]:
class Runnable:
    def __init__(self, func):
        self.func = func
    def __or__(self, other):
        def chained_func(*args, **kwargs):
            print(f"Chaining {self.func.__name__} | {other.func.__name__}")
            print(f"  with args={args}, kwargs={kwargs}")
            return other.invoke(self.func(*args, **kwargs))
        return Runnable(chained_func)
    def invoke(self, *args, **kwargs):
        print(f"Invoking {self.func.__name__} with args={args}, kwargs={kwargs}")
        return self.func(*args, **kwargs)

With the `Runnable` class, we will be able wrap a function into the class, allowing us to then chain together multiple of these _runnable_ functions using the `__or__` method.

First, let's create a few functions that we'll chain together:

In [34]:
def add_five(x):
    print(f"add_five({x})")
    return x+5

def sub_five(x):
    print(f"sub_five({x})")
    return x-5

def mul_five(x):
    print(f"mul_five({x})")
    return x*5

Now we wrap our functions with the `Runnable`:

In [35]:
add_five_runnable = Runnable(add_five)
sub_five_runnable = Runnable(sub_five)
mul_five_runnable = Runnable(mul_five)

Finally, we can chain these together using the `__or__` method from the `Runnable` class:

In [36]:
chain = (add_five_runnable).__or__(sub_five_runnable).__or__(mul_five_runnable)

chain.invoke(3)

Invoking chained_func with args=(3,), kwargs={}
Chaining chained_func | mul_five
  with args=(3,), kwargs={}
Chaining add_five | sub_five
  with args=(3,), kwargs={}
add_five(3)
Invoking sub_five with args=(8,), kwargs={}
sub_five(8)
Invoking mul_five with args=(3,), kwargs={}
mul_five(3)


15

So we can see that we're able to chain together our functions using `__or__`. The pipe `|` operator is simply a shortcut for the `__or__` method, so we can create the exact same chain like so:

In [33]:
chain = add_five_runnable | sub_five_runnable | mul_five_runnable

chain.invoke(3)

Invoking chained_func with args=(3,), kwargs={}
Chaining chained_func | mul_five
  with args=(3,), kwargs={}
Chaining add_five | sub_five
  with args=(3,), kwargs={}
Invoking sub_five with args=(8,), kwargs={}
Invoking mul_five with args=(3,), kwargs={}


15

## LCEL `RunnableLambda`

The `RunnableLambda` class is LangChain's built-in method for constructing a _runnable_ object from a function. That is, it does the same thing as the custom `Runnable` class we created earlier. Let's try it out with the same functions as before.

In [37]:
from langchain_core.runnables import RunnableLambda

add_five_runnable = RunnableLambda(add_five)
sub_five_runnable = RunnableLambda(sub_five)
mul_five_runnable = RunnableLambda(mul_five)

We chain these together again with the pipe `|` operator:

In [38]:
chain = add_five_runnable | sub_five_runnable | mul_five_runnable

And call them using the `invoke` method:

In [39]:
chain.invoke(3)

add_five(3)
sub_five(8)
mul_five(3)


15

Now we want to try something a little more testing, so this time we will generate a report, and we will try and edit that report using this functionallity.

In [40]:
prompt_str = "give me a small report about {topic}"
prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_str
)

In [41]:
chain = prompt | llm | output_parser

In [42]:
result = chain.invoke("AI")
display(Markdown(result))

### Report on Artificial Intelligence (AI)

#### Introduction
Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn like humans. It encompasses a variety of technologies, including machine learning, natural language processing, robotics, and computer vision. AI has rapidly evolved over the past few decades, transforming industries and daily life.

#### Current State of AI
As of 2023, AI technologies are integrated into numerous applications, including:

1. **Healthcare**: AI is used for diagnostics, personalized medicine, and predictive analytics. Machine learning algorithms analyze medical data to identify patterns and assist in treatment plans.

2. **Finance**: AI algorithms are employed for fraud detection, algorithmic trading, and risk management. They analyze vast amounts of data to make real-time decisions.

3. **Transportation**: Autonomous vehicles utilize AI for navigation and safety. AI systems process data from sensors to make driving decisions.

4. **Customer Service**: Chatbots and virtual assistants powered by AI provide 24/7 customer support, handling inquiries and resolving issues efficiently.

5. **Manufacturing**: AI optimizes supply chains, predicts maintenance needs, and enhances production processes through automation.

#### Challenges and Ethical Considerations
Despite its benefits, AI poses several challenges:

- **Bias and Fairness**: AI systems can perpetuate existing biases present in training data, leading to unfair outcomes in areas like hiring and law enforcement.

- **Privacy Concerns**: The use of AI in data collection raises significant privacy issues, as personal information can be misused or inadequately protected.

- **Job Displacement**: Automation driven by AI may lead to job losses in certain sectors, necessitating workforce retraining and adaptation.

- **Accountability**: Determining responsibility for decisions made by AI systems, especially in critical areas like healthcare and autonomous driving, remains a complex issue.

#### Future Trends
The future of AI is promising, with several trends expected to shape its development:

- **Explainable AI**: There is a growing demand for AI systems that can provide transparent and understandable reasoning behind their decisions.

- **AI in Creative Fields**: AI is increasingly being used in art, music, and content creation, leading to new forms of collaboration between humans and machines.

- **Regulation and Governance**: As AI becomes more pervasive, governments and organizations are likely to implement regulations to ensure ethical use and mitigate risks.

- **Human-AI Collaboration**: The focus will shift towards enhancing human capabilities through AI, fostering collaboration rather than replacement.

#### Conclusion
Artificial Intelligence is a transformative technology with the potential to revolutionize various sectors. While it offers significant benefits, addressing the associated challenges and ethical considerations is crucial for its responsible development and deployment. As AI continues to evolve, its impact on society will depend on how we navigate these complexities.

Here we are making two functions, `extract_fact` to pull out the main content of our text and `replace_word` that will replace AI with Skynet!

In [43]:
def extract_fact(x):
    if "\n\n" in x:
        return "\n".join(x.split("\n\n")[1:])
    else:
        return x

old_word = "AI"
new_word = "skynet"

def replace_word(x):
    return x.replace(old_word, new_word)

Lets wrap these functions and see what the output is!

In [44]:
extract_fact_runnable = RunnableLambda(extract_fact)
replace_word_runnable = RunnableLambda(replace_word)

In [61]:
chain = prompt | llm | output_parser | extract_fact_runnable | replace_word_runnable

In [45]:
result = chain.invoke("retrieval augmented generation")
display(Markdown(result))

### Report on Retrieval-Augmented Generation (RAG)

#### Introduction
Retrieval-Augmented Generation (RAG) is an innovative approach that combines the strengths of information retrieval and natural language generation. This method enhances the capabilities of language models by allowing them to access external knowledge sources, thereby improving the accuracy and relevance of generated responses.

#### Concept Overview
RAG operates on the principle of integrating a retrieval mechanism with a generative model. The process typically involves two main components:

1. **Retrieval Component**: This part of the system retrieves relevant documents or pieces of information from a large corpus based on a given query. It uses techniques such as vector embeddings and similarity search to identify the most pertinent data.

2. **Generation Component**: After retrieving the relevant information, the generative model (often based on architectures like Transformers) synthesizes a coherent and contextually appropriate response. This model leverages the retrieved data to enhance its output, ensuring that the generated text is not only fluent but also factually grounded.

#### Advantages
- **Enhanced Accuracy**: By accessing up-to-date and domain-specific information, RAG can produce more accurate and contextually relevant responses compared to traditional generative models that rely solely on pre-existing training data.
- **Dynamic Knowledge Integration**: RAG allows for the incorporation of new information without the need for retraining the entire model, making it adaptable to changing knowledge landscapes.
- **Improved Contextual Understanding**: The retrieval process helps the model to better understand the context of the query, leading to more nuanced and informed responses.

#### Applications
RAG has a wide range of applications, including:
- **Question Answering**: Providing precise answers to user queries by retrieving relevant documents and generating responses based on that information.
- **Chatbots and Virtual Assistants**: Enhancing conversational agents with the ability to pull in real-time data, making interactions more informative and engaging.
- **Content Creation**: Assisting writers and content creators by providing relevant information and suggestions based on current trends and data.

#### Challenges
Despite its advantages, RAG faces several challenges:
- **Quality of Retrieved Information**: The effectiveness of the generation depends heavily on the quality and relevance of the retrieved documents. Poor retrieval can lead to inaccurate or misleading outputs.
- **Computational Complexity**: The dual process of retrieval and generation can be resource-intensive, requiring significant computational power and optimization.
- **Integration of Diverse Sources**: Ensuring that the retrieved information is coherent and seamlessly integrated into the generated text can be complex, especially when dealing with conflicting data.

#### Conclusion
Retrieval-Augmented Generation represents a significant advancement in the field of natural language processing, merging the capabilities of information retrieval with generative models. As research and development in this area continue to evolve, RAG is poised to enhance various applications, making AI systems more knowledgeable, responsive, and context-aware. Future work will likely focus on improving retrieval accuracy, reducing computational demands, and refining the integration of diverse information sources.

Those are our `RunnableLambda` functions. It's worth noting that all inputs to these functions are expected to be a SINGLE arguments. If you have a function that accepts multiple arguments, you can input a dictionary with keys, then unpack them inside the function.

## LCEL `RunnableParallel` and `RunnablePassthrough`

LCEL provides us with various `Runnable` classes that allow us to control the flow of data and execution order through our chains. Two of these are `RunnableParallel` and `RunnablePassthrough`.

* `RunnableParallel` — allows us to run multiple `Runnable` instances in parallel. Acting almost as a Y-fork in the chain.

* `RunnablePassthrough` — allows us to pass through a variable to the next `Runnable` without modification.

To see these runnables in action, we will create two data sources, each source provides specific information but to answer the question we will need both to fed to the LLM.

In [46]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

embedding = OpenAIEmbeddings()

vecstore_a = DocArrayInMemorySearch.from_texts(
    [
        "half the info is here",
        "DeepSeek-V3 was released in December 2024"
    ],
    embedding=embedding
)
vecstore_b = DocArrayInMemorySearch.from_texts(
    [
        "the other half of the info is here",
        "the DeepSeek-V3 LLM is a mixture of experts model with 671B parameters"
    ],
    embedding=embedding
)

C:\Users\karol\AppData\Local\Temp\ipykernel_10636\3699332964.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()
d:\Karol\work\Techniczne\Szkolenia\AurelioLabs_Langchain\.venv\Lib\site-packages\docarray\helper.py:255: SyntaxWarning: invalid escape sequence '\*'
  e.g. '\*.py', '[\*.zip, \*.gz]'
d:\Karol\work\Techniczne\Szkolenia\AurelioLabs_Langchain\.venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


Here you can see the prompt does have three inputs, two for context and one for the question itself.

In [47]:
prompt_str = """Using the context provided, answer the user's question.
Context:
{context_a}
{context_b}
"""

In [48]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prompt_str),
    HumanMessagePromptTemplate.from_template("{question}")
])

Here we are wrapping our vector stores as retrievers so they can be fitted into one big retrieval variable to be used by the prompt.

In [49]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

retrieval = RunnableParallel(
    {
        "context_a": retriever_a, "context_b": retriever_b, "question": RunnablePassthrough()
    }
)

The chain we'll be constructing will look something like this:

![](https://github.com/aurelio-labs/langchain-course/blob/main/assets/lcel-flow.png?raw=1)

In [50]:
chain = retrieval | prompt | llm | output_parser

We `invoke` it as usual.

In [51]:
result = chain.invoke(
    "what architecture does the model DeepSeek released in december use?"
)
result

'The DeepSeek-V3 model, released in December 2024, is a mixture of experts model with 671 billion parameters.'

With that we've seen how we can use `RunnableParallel` and `RunnablePassthrough` to control the flow of data and execution order through our chains.

---